In [12]:
#last run 02/05/2024 0541
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import transforms, datasets
from torchsummary import summary
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [13]:
import os

In [14]:
if __name__ == "__main__":
    files = os.path.join("yout file name")

In [15]:
#Defining the transformations to apply on our dataset
transformations = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

In [16]:
# Load your dataset
if __name__ == "__main__":
    dataset = datasets.ImageFolder(root='your file name', transform=transformations)

In [17]:
# Define the sizes of your splits
if __name__ == "__main__":
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size

In [18]:
# Randomly split the dataset into train and test
if __name__ == "__main__":
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [19]:
# Create DataLoaders for your train and test datasets
if __name__ == "__main__":
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [20]:
class SimpleCNN(nn.Module):
    def __init__(self, in_channels):
        super(SimpleCNN, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        return x

# Define DeepConcat layer
class DeepConcat(nn.Module):
    def __init__(self):
        super(DeepConcat, self).__init__()

    def forward(self, inputs):
        return torch.cat(inputs, dim=1)

# Define the architecture with three "hands"
class ThreeHandsArchitecture(nn.Module):
    def __init__(self, num_classes=7):
        super(ThreeHandsArchitecture, self).__init__()
        self.hand1 = SimpleCNN(in_channels=3)
        self.hand2 = nn.Sequential(
            SimpleCNN(in_channels=3),  # Adjusted in_channels based on the output of the first core structure
            SimpleCNN(in_channels=32)   # Adjusted in_channels based on the output of the second core structure
        )
        self.hand3 = nn.Sequential(
            SimpleCNN(in_channels=3), 
            SimpleCNN(in_channels=32),  # Adjusted in_channels based on the output of the first core structure
            SimpleCNN(in_channels=32)   # Adjusted in_channels based on the output of the second core structure
        )
        self.deep_concat = DeepConcat()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(58*58*32*3, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        hand1_output = self.hand1(x)
        hand2_output_1 = self.hand2[0](x)
        hand2_output_2 = self.hand2[1](hand2_output_1)
        hand3_output_1 = self.hand3[0](x)
        hand3_output_2 = self.hand3[1](hand3_output_1)
        hand3_output_3 = self.hand3[2](hand3_output_2)
        
        # Resize outputs to a common size (assuming hand1_output is the reference size)
        common_size = (hand1_output.size()[2], hand1_output.size()[3])
        hand1_output = nn.functional.interpolate(hand3_output_1, size=common_size)
        hand2_output_2 = nn.functional.interpolate(hand3_output_2, size=common_size)
        hand3_output_3 = nn.functional.interpolate(hand3_output_3, size=common_size)
        
        concatenated_output = self.deep_concat([hand1_output, hand2_output_2, hand3_output_3])
        flattened_output = self.flatten(concatenated_output)
        output = self.fc(flattened_output)
        output = self.softmax(output)
        return output

In [65]:
if __name__ == "__main__":
    model = ThreeHandsArchitecture()

In [84]:
#Defing the hyperparameters
num_epochs = 20
learning_rate = 0.005

In [85]:
# Define loss function and optimizer
if __name__ == "__main__":
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,weight_decay=0.0001)

In [ ]:
# Training loop
if __name__ == "__main__":
    loss_values = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}/{num_epochs}")
        iterations = 0
        acc_loss = 0
        for inputs, labels in loop:
            inputs, labels = inputs, labels
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            iterations += 1
            acc_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        loss_value = acc_loss/iterations
        loss_values.append(loss_value)
        print(f'\nEpoch [{epoch+1}/{num_epochs}], Loss: {loss_value:.4f}')

In [ ]:
# Plotting the loss values
if __name__ == "__main__":
    plt.plot(loss_values, label='Training Loss')
    plt.title('Loss Function vs. Iteration')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
# Testing our model
if __name__ == "__main__":
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}/{num_epochs}")
        # for inputs, labels in test_loader:
        for inputs, labels in loop:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loop.set_postfix()
    accuracy = correct / total


In [ ]:
if __name__ == "__main__":
    print(f'\nTrain Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Testing our model
if __name__ == "__main__":
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        loop = tqdm(test_loader, leave=True, desc=f"Epoch {epoch+1}/{num_epochs}")
        # for inputs, labels in test_loader:
        for inputs, labels in loop:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loop.set_postfix()
    accuracy = correct / total


In [ ]:
if __name__ == "__main__":
    print(f'\nTest Accuracy: {accuracy * 100:.2f}%')

In [95]:
#torch.save(model.state_dict(), 'your_folder_location/three_hands_model.pth')